In [46]:
import pandas as pd
import tensorflow as tf


In [47]:
test_label = pd.read_csv ('data/processed/model1/test_y_split.csv')
g = pd.read_pickle ('data/interim/X_group.pkl')
t = pd.read_pickle ('data/interim/X_technique.pkl')
t =t[['technique_ID', 'input_technique_tactics']]
tactics_order = pd.read_csv ('data/raw/tactics_order.csv', index_col=0)
tactics_order

,tactic_ID,tactic_name,stage_order
0,TA0043,reconnaissance,0
1,TA0042,resource_development,0
2,TA0001,initial_access,1
3,TA0002,execution,2
4,TA0003,persistence,3
5,TA0004,privilege_escalation,4
6,TA0005,defense_evasion,5
7,TA0006,credential_access,6
8,TA0007,discovery,7
9,TA0008,lateral_movement,8


In [48]:
t_tactic_stage = pd.merge (
    left = t.explode('input_technique_tactics'),
    right = tactics_order,
    left_on= 'input_technique_tactics', 
    right_on= 'tactic_name', 
    how = 'left'
)
t_tactic_stage = t_tactic_stage.groupby ('technique_ID', as_index= False).agg(list)
t_tactic_stage['earliest_stage'] = t_tactic_stage['stage_order'].apply(min)
t_tactic_stage['lastest_stage'] = t_tactic_stage['stage_order'].apply(max)
t_tactic_stage.drop (columns = ['input_technique_tactics', 'tactic_name','stage_order', 'tactic_ID'], inplace= True)



,technique_ID,earliest_stage,lastest_stage
0,T1001,10,10
1,T1001.001,10,10
2,T1001.002,10,10
3,T1001.003,10,10
4,T1003,6,6
...,...,...,...
602,T1648,2,2
603,T1649,6,6
604,T1650,0,0
605,T1651,2,2


In [49]:
test_label

,group_ID,technique_ID,label
0,G0007,T1548.002,0.0
1,G0007,T1134,0.0
2,G0007,T1134.002,0.0
3,G0007,T1134.001,1.0
4,G0007,T1531,0.0
...,...,...,...
5028,G0102,T1102.002,0.0
5029,G0102,T1102.001,0.0
5030,G0102,T1102.003,0.0
5031,G0102,T1047,1.0


In [50]:
test_label = pd.merge (left =test_label, right = t_tactic_stage, on = 'technique_ID', how = 'left')
test_label = test_label[test_label['earliest_stage'] >=1]

In [51]:
test_label

,group_ID,technique_ID,label,earliest_stage,lastest_stage
0,G0007,T1548.002,0.0,4,5
1,G0007,T1134,0.0,4,5
2,G0007,T1134.002,0.0,4,5
3,G0007,T1134.001,1.0,4,5
4,G0007,T1531,0.0,12,12
...,...,...,...,...,...
5028,G0102,T1102.002,0.0,10,10
5029,G0102,T1102.001,0.0,10,10
5030,G0102,T1102.003,0.0,10,10
5031,G0102,T1047,1.0,2,2


In [15]:
g_tactic = pd.read_csv ('data/interim/input_group_tactics_vocab.csv')
t_tactic = pd.read_csv ('data/interim/input_technique_tactics_vocab.csv')

In [17]:
t_tactic

,defense_evasion
0,privilege_escalation
1,impact
2,discovery
3,persistence
4,resource_development
5,reconnaissance
6,collection
7,credential_access
8,command_and_control
9,exfiltration


In [1]:
import tensorflow as tf

# Example variable-length lists of fixed-length embeddings for 3 users
user_embeddings = [
    [[1.0, 2.0, 3.0], [2.0, 3.0, 4.0], [3.0, 4.0, 5.0]],  # User 1 with 3 embeddings
    [[4.0, 5.0, 6.0], [5.0, 6.0, 7.0]],                 # User 2 with 2 embeddings
    [[7.0, 8.0, 9.0]]                                    # User 3 with 1 embedding
]

# Convert to TensorFlow constant
user_embeddings_tf = [tf.constant(embedding_list, dtype=tf.float32) for embedding_list in user_embeddings]

# Pad sequences to the same length (if needed)
padded_user_embeddings = tf.keras.preprocessing.sequence.pad_sequences(
    user_embeddings_tf, padding='post', dtype='float32'
)

# Calculate mean along the sequence axis to get fixed-length embeddings for each user
user_avg_embeddings = tf.reduce_mean(padded_user_embeddings, axis=1)

print("Fixed-length embeddings for each user:")
print(user_avg_embeddings.numpy())


Fixed-length embeddings for each user:
[[2.        3.        4.       ]
 [3.        3.6666667 4.3333335]
 [2.3333333 2.6666667 3.       ]]
